In [47]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score
from pathlib import Path
import torch
from transformers import (
    BertTokenizer,
    BertModel,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from torch.utils.data import DataLoader
import tqdm.notebook as tq
from datasets import Dataset
from sklearn.preprocessing import OneHotEncoder

In [48]:
from torch import cuda

device = "cuda" if cuda.is_available() else "cpu"

## Corpus

In [49]:
### Load corpus

data_path = Path("data/MELD_train_efr.json")
assert data_path.exists(), "Data file is not present"
df = pd.read_json(data_path, dtype={"speakers": np.array})  # , "triggers": np.array})
EPISODE, SPEAKERS, EMOTIONS, UTTERANCES, TRIGGERS = df.columns

#### Data exploration

In [50]:
df.head()

,episode,speakers,emotions,utterances,triggers
0,utterance_0,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise]",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 1.0, 0.0]"
1,utterance_1,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
2,utterance_2,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
3,utterance_3,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,utterance_4,"[Joey, Rachel, Joey, Rachel]","[surprise, sadness, surprise, fear]",[But then who? The waitress I went out with la...,"[0.0, 0.0, 1.0, 0.0]"


In [51]:
df.describe()

,episode,speakers,emotions,utterances,triggers
count,4000,4000,4000,4000,4000
unique,4000,3350,3427,3998,523
top,utterance_0,"[Monica, Chandler, Monica]","[neutral, neutral, joy]",[Happy?! Is that what I'm supposed to be Vic? ...,"[0.0, 1.0, 0.0]"
freq,1,15,30,2,191


In [52]:
### Look for how many groups of episodes with the same first utterance there are and their lenghts

df.sort_values(by=UTTERANCES, inplace=True)
groups = np.zeros((850,), dtype=int)

index = 0
count = 1
for i in range(1, len(df[UTTERANCES])):
    if df[UTTERANCES][i][0] == df[UTTERANCES][i - 1][0]:
        ### still in the same group
        count += 1
    else:
        ### found new group
        groups[index] = count
        index += 1
        count = 1

groups = groups[groups != 0]

print(f"Number of groups: {len(groups)}")
print(f"Avg group len: {np.average(groups):.1f}")
print(f"Longest group: {np.max(groups)}")
print(f"Episodes not in a group: {groups[groups == 1].shape[0]}")

Number of groups: 832
Avg group len: 4.8
Longest group: 16
Episodes not in a group: 128


In [53]:
### Count how many speakers there are in each episode

speakers_count = df[SPEAKERS].apply(lambda arr: np.unique(arr).shape[0]).to_numpy()
min_sp = np.min(speakers_count)
max_sp = np.max(speakers_count)
print("Distribution of number of speakers:")
for count in range(min_sp, max_sp + 1):
    print(f"{count} speakers:  {np.sum(speakers_count == count)}")

Distribution of number of speakers:
1 speakers:  214
2 speakers:  2105
3 speakers:  1030
4 speakers:  405
5 speakers:  161
6 speakers:  74
7 speakers:  10
8 speakers:  1


#### Data cleanup

In [54]:
### Drop not useful column

df.drop(columns=[SPEAKERS], inplace=True)

In [55]:
### Remove Nones from the triggers

df[TRIGGERS] = df[TRIGGERS].apply(
    lambda trig_seq: np.array([0.0 if t is None else t for t in trig_seq])
)

In [56]:
df["utterances"][0]

["also I was the point person on my company's transition from the KL-5 to GR-6 system.",
 "You must've had your hands full.",
 'That I did. That I did.',
 "So let's talk a little bit about your duties.",
 'My duties?  All right.']

### Data preprocessing:

explode the dataframe: each rows contains previous utterance, target utterance, next utterance (for context) 

In [57]:
df.head(1)

,episode,emotions,utterances,triggers
1061,utterance_1061,"[joy, neutral, surprise]","[""Happy birthday to you!"", You're paying for t...","[0.0, 1.0, 0.0]"


In [58]:
### flatten the lists of utterances,triggers,emotions into new rows of the dataframe

exploded_df = df.explode([UTTERANCES, TRIGGERS, EMOTIONS], ignore_index=True)
exploded_df.rename(columns={UTTERANCES: "current"}, inplace=True)
exploded_df.head(10)

,episode,emotions,current,triggers
0,utterance_1061,joy,"""Happy birthday to you!""",0.0
1,utterance_1061,neutral,You're paying for that.,1.0
2,utterance_1061,surprise,What? No-no it's her birthday!,0.0
3,utterance_1062,joy,"""Happy birthday to you!""",0.0
4,utterance_1062,neutral,You're paying for that.,0.0
5,utterance_1062,surprise,What? No-no it's her birthday!,1.0
6,utterance_1062,surprise,You've sung Happy Birthday to 20 different wom...,0.0
7,utterance_1063,joy,"""Happy birthday to you!""",0.0
8,utterance_1063,neutral,You're paying for that.,0.0
9,utterance_1063,surprise,What? No-no it's her birthday!,0.0


In [60]:
### Pair 2 shifted columns of utterance to the exploded df

next = pd.concat((exploded_df["current"][1:], pd.Series([" "]))).to_list()
previous = pd.concat((pd.Series([" "]), exploded_df["current"][:-1])).to_list()

exploded_df.insert(loc=2, column="previous", value=previous)
exploded_df.insert(loc=4, column="next", value=next)

exploded_df.head(10)

,episode,emotions,previous,current,next,triggers
0,utterance_1061,joy,,"""Happy birthday to you!""",You're paying for that.,0.0
1,utterance_1061,neutral,"""Happy birthday to you!""",You're paying for that.,What? No-no it's her birthday!,1.0
2,utterance_1061,surprise,You're paying for that.,What? No-no it's her birthday!,"""Happy birthday to you!""",0.0
3,utterance_1062,joy,What? No-no it's her birthday!,"""Happy birthday to you!""",You're paying for that.,0.0
4,utterance_1062,neutral,"""Happy birthday to you!""",You're paying for that.,What? No-no it's her birthday!,0.0
5,utterance_1062,surprise,You're paying for that.,What? No-no it's her birthday!,You've sung Happy Birthday to 20 different wom...,1.0
6,utterance_1062,surprise,What? No-no it's her birthday!,You've sung Happy Birthday to 20 different wom...,"""Happy birthday to you!""",0.0
7,utterance_1063,joy,You've sung Happy Birthday to 20 different wom...,"""Happy birthday to you!""",You're paying for that.,0.0
8,utterance_1063,neutral,"""Happy birthday to you!""",You're paying for that.,What? No-no it's her birthday!,0.0
9,utterance_1063,surprise,You're paying for that.,What? No-no it's her birthday!,You've sung Happy Birthday to 20 different wom...,0.0


In [61]:
for i in range(1, len(exploded_df) - 1):
    if exploded_df[EPISODE][i] != exploded_df[EPISODE][i - 1]:
        exploded_df["next"][i - 1] = " "
        exploded_df["previous"][i] = " "

In [62]:
exploded_df.head(30)

,episode,emotions,previous,current,next,triggers
0,utterance_1061,joy,,"""Happy birthday to you!""",You're paying for that.,0.0
1,utterance_1061,neutral,"""Happy birthday to you!""",You're paying for that.,What? No-no it's her birthday!,1.0
2,utterance_1061,surprise,You're paying for that.,What? No-no it's her birthday!,,0.0
3,utterance_1062,joy,,"""Happy birthday to you!""",You're paying for that.,0.0
4,utterance_1062,neutral,"""Happy birthday to you!""",You're paying for that.,What? No-no it's her birthday!,0.0
5,utterance_1062,surprise,You're paying for that.,What? No-no it's her birthday!,You've sung Happy Birthday to 20 different wom...,1.0
6,utterance_1062,surprise,What? No-no it's her birthday!,You've sung Happy Birthday to 20 different wom...,,0.0
7,utterance_1063,joy,,"""Happy birthday to you!""",You're paying for that.,0.0
8,utterance_1063,neutral,"""Happy birthday to you!""",You're paying for that.,What? No-no it's her birthday!,0.0
9,utterance_1063,surprise,You're paying for that.,What? No-no it's her birthday!,You've sung Happy Birthday to 20 different wom...,0.0


In [ ]:
exploded_df.iloc[0]


If an episode contains the same utterances of the previous and a few more then the triggers from the previous episode are replicated in the current episode

In [ ]:
for i in range(10):
    print(f"{df[TRIGGERS][i]}")

In [ ]:
count = 0
for i in range(1, len(df)):
    is_continuation = np.all([u in df[UTTERANCES][i] for u in df[UTTERANCES][i - 1]])
    if is_continuation:
        count += 1
        for k, t in enumerate(df[TRIGGERS][i - 1]):
            df[TRIGGERS][i][k] = t

In [ ]:
for i in range(10):
    print(f"{df[TRIGGERS][i]}")

#### Train/Validation/Test split

In [ ]:
### Train Val Test split: 80/10/10


def split_data(df: pd.DataFrame, seed: int = 42):
    df_train, df_test = train_test_split(
        df, test_size=0.2, train_size=0.8, random_state=seed
    )

    df_val, df_test = train_test_split(
        df_test, test_size=0.5, train_size=0.5, random_state=seed
    )

    return df_train, df_val, df_test

In [ ]:
### Check
df_train, df_val, df_test = split_data(df)
print(f"df_train len: {len(df_train)}")
print(f"df_val len: {len(df_val)}")
print(f"df_test len: {len(df_test)}")

In [ ]:
### Class imbalance check
classes_count = {}
for emotions in df_train["emotions"]:
    for emotion in emotions:
        if emotion in classes_count:
            classes_count[emotion] += 1
        else:
            classes_count[emotion] = 1

### then we sort the dictionary by occurences
emotions_dict = {
    k: v
    for k, v in sorted(classes_count.items(), key=lambda item: item[1], reverse=True)
}
print("Classes values:")
print(emotions_dict)

# Class imbalance abbastanza alto, potremmo usare dei weights

### Metrics

In [ ]:
### given 2 series of sequences of triggers/emotions compute F1 inside each sequence and return avg, flatten out and compute F1
def sequence_f1(y_true, y_pred, avg: bool = True):
    res = [
        f1_score(y_true=y_t, y_pred=y_p, average="micro")
        for y_t, y_p in zip(y_true, y_pred)
    ]
    return np.average(res) if avg else res


def unrolled_f1(y_true, y_pred):
    y_t_flat = []
    for l in y_true:
        for e in l:
            y_t_flat.append(e)

    y_p_flat = []
    for l in y_pred:
        for e in l:
            y_p_flat.append(e)

    return f1_score(y_true=y_t_flat, y_pred=y_p_flat, average="micro")

## Baseline Models: 

In [ ]:
### Create baseline models


class SequenceDummyClassifier(DummyClassifier):
    def __init__(self, strategy: str, seed: int = 42) -> None:
        self.seed = seed
        if not strategy.lower() in ("random", "majority"):
            raise ValueError("strategy must be in [random, majority]")
        sklearn_strategy = "uniform" if strategy == "random" else "most_frequent"
        super().__init__(strategy=sklearn_strategy, random_state=seed)

    def _flatten_seq_grouping(self, df: pd.Series):
        lens = df.apply(lambda s: len(s)).to_numpy()
        max_len = np.max(lens)
        min_len = np.min(lens)
        res = np.empty((0,))
        for group_len in range(min_len, max_len + 1):
            group = df.loc[lens == group_len]
            np_group = np.array(group.tolist()).flatten()
            res = np.hstack((res, np_group))
        return res

    def _flatten_seq(self, df: pd.Series):
        res = []
        for l in df:
            for e in l:
                res.append(e)
        return res

    def _deflatten_seq(self, seq, shape_like: pd.Series):
        data = iter(seq)
        result = [[next(data) for _ in s] for s in shape_like]
        return result

    def fit(self, X: pd.Series, y: pd.Series):
        X_flat = self._flatten_seq(X)
        y_flat = self._flatten_seq(y)
        super().fit(X=X_flat, y=y_flat)

    def predict(self, X: pd.Series, return_flat: bool = False):
        X_flat = self._flatten_seq(X)
        y_flat = super().predict(X_flat)
        return y_flat if return_flat else self._deflatten_seq(seq=y_flat, shape_like=X)

In [ ]:
def experiment_baseline(df_train: pd.DataFrame, df_test: pd.DataFrame, seed: int = 42):

    baseline_f1s = {}
    baseline_results = {}

    for strategy in ("Random", "Majority"):
        for target in (EMOTIONS, TRIGGERS):
            clf = SequenceDummyClassifier(strategy=strategy, seed=seed)
            clf.fit(X=df_train[UTTERANCES], y=df_train[target])

            res = clf.predict(X=df_test[UTTERANCES], return_flat=False)
            baseline_results.update({f"{target}_{strategy}": res})

            seq_f1 = sequence_f1(y_true=df_test[target], y_pred=res)
            baseline_f1s.update({f"sequence_f1({target}_{strategy})": seq_f1})

            unr_f1 = unrolled_f1(y_true=df_test[target], y_pred=res)
            baseline_f1s.update({f"unrolled_f1({target}_{strategy})": unr_f1})

    return baseline_f1s, baseline_results


f1s, results = experiment_baseline(df_train, df_test)
for k, v in f1s.items():
    print(f"{k} : {v}")

# Tokenization

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# TODO discuss: questo fa schifo
# encoder = OneHotEncoder(handle_unknown="error", sparse_output=False)
all_emotions = []
for seq in df_train[EMOTIONS]:
    for e in seq:
        all_emotions.append(e)
uniq_emotions = np.sort(np.unique(all_emotions))
one_hot = np.identity(len(uniq_emotions))
emotion_mapping = {e: one_hot[i] for i, e in enumerate(uniq_emotions)}
print(emotion_mapping)

In [ ]:
def tokenize(ds_row, tokenizer=tokenizer):

    if type(ds_row["emotions"][0]) != str:  ### batchsize > 1
        emotion_encoding = []
        emotion_encoding = [
            [emotion_mapping[e] for e in emotions_of_one_utterance]
            for emotions_of_one_utterance in ds_row["emotions"]
        ]
    else:  ### batchsize == 1
        emotion_encoding = [emotion_mapping[e] for e in ds_row["emotions"]]

    encoded_ds_row = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        # "emotions": ds_row["emotions"],
        "emotions": emotion_encoding,
        "triggers": ds_row["triggers"],
    }

    for sentence in ds_row["utterances"]:

        tokenized_sentence = tokenizer(
            sentence,
            truncation=True,
            padding="max_length",
            max_length=tokenizer.model_max_length // 4,
            return_tensors="pt",
            # is_split_into_words=True,
        )
        encoded_ds_row["input_ids"].append(tokenized_sentence["input_ids"])
        encoded_ds_row["token_type_ids"].append(tokenized_sentence["token_type_ids"])
        encoded_ds_row["attention_mask"].append(tokenized_sentence["attention_mask"])

    return encoded_ds_row

In [ ]:
ds_train = Dataset.from_pandas(df_train)
ds_val = Dataset.from_pandas(df_val)
ds_test = Dataset.from_pandas(df_test)

In [ ]:
### Apply tokenization
batched = True
ds_train_tokenized = ds_train.map(
    function=tokenize,
    fn_kwargs={"tokenizer": tokenizer},
    batched=batched,
    remove_columns=[UTTERANCES],
)
ds_train_tokenized.set_format(type="torch")
ds_val_tokenized = ds_val.map(
    function=tokenize,
    fn_kwargs={"tokenizer": tokenizer},
    batched=batched,
    remove_columns=[UTTERANCES],
)
ds_val_tokenized.set_format(type="torch")
ds_test_tokenized = ds_test.map(
    function=tokenize,
    fn_kwargs={"tokenizer": tokenizer},
    batched=batched,
    remove_columns=[UTTERANCES],
)
ds_test_tokenized.set_format(type="torch")

In [ ]:
print(ds_train_tokenized[0]["input_ids"])

In [ ]:
print(ds_train_tokenized[0])

In [ ]:
print(ds_train[UTTERANCES][0])

In [ ]:
tokenized_row = tokenize(ds_train[0])
# print(tokenized_row["input_ids"][0])
print(tokenizer.batch_decode(tokenized_row["input_ids"][0]))
print(tokenizer.batch_decode(tokenized_row["input_ids"][1]))

In [ ]:
# Test della funzione
tokens = tokenize(ds_train[0], tokenizer)
print(tokens)

# Bert Models

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self, num_emotions=7):
        super(BERTClass, self).__init__()

        self.bert = BertModel.from_pretrained("bert-base-uncased")

        self.dropout = torch.nn.Dropout(0.3)
        # self.lstm = torch.nn.LSTM()
        # classifiers
        self.l_emotions = torch.nn.Linear(self.bert.config.hidden_size, num_emotions)
        self.l_triggers = torch.nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, ids, mask, token_type_ids):
        print(ids.shape)
        print(ids.reshape(-1, ids.size(-1)))
        output = self.bert(ids, attention_mask=mask)  # , token_type_ids=token_type_ids)

        output_emotions = self.dropout(output)
        output_triggers = self.dropout(output)

        # output_triggers = self.lstm(output_triggers)

        output_emotions = self.l_emotions(output_emotions)
        output_triggers = self.l_triggers(output_triggers)

        return output_emotions, output_triggers

    def freeze_params(self):
        for param in self.bert.parameters():
            param.requires_grad = False

# Training Utils

In [ ]:
num_emotions = df["emotions"].explode().nunique()

model_frozen = BERTClass(num_emotions)
model_full = BERTClass(num_emotions)

model_frozen.freeze_params()

# Verifying that the params are actually frozen
for name, param in model_frozen.named_parameters():
    print(name, param.requires_grad)

for name, param in model_full.named_parameters():
    print(name, param.requires_grad)

In [ ]:
model_list = [model_full, model_frozen]
num_epochs = 5

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

for model in model_list:
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
    loss_fn = torch.nn.CrossEntropyLoss()
    model.train()

    # Tokenizer initiation
    # TODO Check tokenizer parameters
    # TODO A Dataloader seems to be commonly used for this use cases
    # encoding = tokenizer(df_train, truncation=False, padding='max_length', return_tensors='pt')
    # input_ids = encoding['input_ids']
    # attention_mask = encoding['attention_mask']

    # Training loop
    # for epoch in range(num_epochs):
    #
    #    for idx in range(len(df_test)):

    #        text = df_train[idx].drop(TRIGGERS)
    #        label = df_train[idx][TRIGGERS]

    #        optimizer.zero_grad()

    #        logits = model(batch_data)
    #        loss = loss_fn(logits, batch_labels)
    #        loss.backward()
    #        optimizer.step()

In [ ]:
def loss_fn(outputs_emotions, outputs_triggers, emotions_labels, triggers_labels):
    return torch.nn.CrossEntropyLoss(
        outputs_emotions, emotions_labels
    ) + torch.nn.BCELoss(outputs_triggers, triggers_labels)

In [ ]:
### Training of the model
def train_model(train_dl, model, optimizer):
    losses = []
    correct_predictions_emotions = 0
    correct_predictions_triggers = 0
    num_samples_emotions = 0
    num_samples_triggers = 0

    ### activate dropout, batch norm
    model.train()

    ### initialize progress bar
    batches = tq.tqdm(
        enumerate(train_dl), total=len(train_dl), leave=True, colour="steelblue"
    )

    for batch_idx, data in batches:
        ids = data["input_ids"].to(device, dtype=torch.long)
        mask = data["attention_mask"].to(device, dtype=torch.long)
        token_type_ids = data["token_type_ids"].to(device, dtype=torch.long)
        emotions_labels = data["emotions"].to(device, dtype=torch.float)
        triggers_labels = data["triggers"].to(device, dtype=torch.float)
        outputs_emotions, outputs_triggers = model(
            ids, mask, token_type_ids
        )  ### Forward

        loss = loss_fn(
            outputs_emotions, outputs_triggers, emotions_labels, triggers_labels
        )
        losses.append(loss.cpu().detach().numpy())

        ### apply thresh 0.5
        outputs_emotions = (
            torch.sigmoid(outputs_emotions).cpu().detach().numpy().round()
        )
        outputs_triggers = (
            torch.sigmoid(outputs_triggers).cpu().detach().numpy().round()
        )

        emotions_labels = emotions_labels.cpu().detach().numpy()
        triggers_labels = triggers_labels.cpu().detach().numpy()

        correct_predictions_emotions += np.sum(outputs_emotions == emotions_labels)
        correct_predictions_triggers += np.sum(outputs_triggers == triggers_labels)

        num_samples_emotions += emotions_labels.size
        num_samples_triggers += triggers_labels.size

        ### Backward
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        ### Grad descent step
        optimizer.step()

        ### Update progress bar
        batches.set_description(f"")
        batches.set_postfix(batch_loss=loss)

    # Si potrebbe fare una singola accuracy come media delle due, magari fuori dal training
    accuracy_emotions = float(correct_predictions_emotions) / num_samples_emotions
    accuracy_triggers = float(correct_predictions_triggers) / num_samples_triggers

    return model, accuracy_emotions, accuracy_triggers, losses

In [ ]:
# eval model, setup e train_eval da definire

In [ ]:
def eval_model(validation_dl, model):
    losses = []
    correct_predictions = 0
    num_samples = 0
    num_categories = next(iter(validation_dl))["labels"].shape[1]

    ### accumulate data over each batch to compute the f1
    true_positives = np.array([0 for _ in range(num_categories)])
    false_positives = np.array([0 for _ in range(num_categories)])
    false_negatives = np.array([0 for _ in range(num_categories)])

    ### turn off dropout, fix batch norm
    model.eval()

    ### show progress bar
    batches = tq.tqdm(
        enumerate(validation_dl),
        total=len(validation_dl),
        leave=True,
        colour="steelblue",
    )
    with torch.no_grad():
        for batch_idx, data in batches:
            ids = data["input_ids"].to(device, dtype=torch.long)
            mask = data["attention_mask"].to(device, dtype=torch.long)
            token_type_ids = data["token_type_ids"].to(device, dtype=torch.long)
            emotions_labels = data["emotions"].to(device, dtype=torch.float)
            triggers_labels = data["triggers"].to(device, dtype=torch.float)
            outputs_emotions, outputs_triggers = model(ids, mask, token_type_ids)

            loss = loss_fn(
                outputs_emotions, outputs_triggers, emotions_labels, triggers_labels
            )
            losses.append(loss.cpu().detach().numpy())

            ### validation accuracy
            ### training sigmoid is in BCEWithLogitsLoss
            outputs_emotions = (
                torch.sigmoid(outputs_emotions).cpu().detach().numpy().round()
            )
            outputs_triggers = (
                torch.sigmoid(outputs_triggers).cpu().detach().numpy().round()
            )

            emotions_labels = emotions_labels.cpu().detach().numpy()
            triggers_labels = triggers_labels.cpu().detach().numpy()
            correct_predictions_emotions += np.sum(outputs_emotions == emotions_labels)
            correct_predictions_triggers += np.sum(outputs_triggers == triggers_labels)

            num_samples_emotions += emotions_labels.size
            num_samples_triggers += triggers_labels.size

        accuracy_emotions = float(correct_predictions_emotions) / num_samples_emotions
        accuracy_triggers = float(correct_predictions_triggers) / num_samples_triggers
        # precision = true_positives / (true_positives + false_positives)
        # recall = true_positives / (true_positives + false_negatives)
        # f1_per_cat = 2 * (precision * recall) / (precision + recall)
        # f1_overall = np.mean(f1_per_cat)

    return accuracy_emotions, accuracy_triggers, losses  # , f1_overall, f1_per_cat

In [ ]:
def train_eval(
    train_dl,
    validation_dl,
    model,
    optimizer,
    n_epochs=1,
    save_name="0",
    train_model_f=train_model,
    eval_model_f=eval_model,
):
    model_folder = Path.cwd().joinpath("models")
    if not model_folder.exists():
        model_folder.mkdir(parents=True)

    history = {}
    best_f1 = 0

    for epoch in range(1, n_epochs + 1):
        print(f"Epoch {epoch}/{n_epochs}")
        model, accuracy_emotions, accuracy_triggers, train_loss = train_model_f(
            train_dl, model, optimizer
        )
        val_accuracy_emotions, val_accuracy_triggers, val_loss = eval_model_f(
            validation_dl, model
        )

        print(
            f"train_loss={np.mean(train_loss):.4f}, val_loss={np.mean(val_loss):.4f}, "
            f"train_acc_emo={accuracy_emotions:.4f}, train_acc_emo={accuracy_triggers:.4f},"
            f"val_acc_emo={val_accuracy_emotions:.4f}, val_acc_tri={val_accuracy_triggers:.4f}, "
        )

        history.update({"train_acc_emo": accuracy_emotions})
        history.update({"train_acc_tri": accuracy_triggers})
        history.update({"train_losses": train_loss})
        history.update({"val_acc_emo": val_accuracy_emotions})
        history.update({"val_acc_emo": val_accuracy_triggers})
        history.update({"val_losses": val_loss})
        # history.update({"f1_overall": f1_overall})
        # history.update({"f1_per_cat": f1_per_cat})

        ### save the best model
        # if f1_overall > best_f1:
        #   torch.save(
        #      model.state_dict(),
        #      Path.joinpath(model_folder, f"model_{save_name}.bin"),
        # )
        # best_f1 = f1_overall

    return history  # (history["f1_overall"], history["f1_per_cat"], history["train_losses"])

In [ ]:
def create_data_loaders(tokenized_datasets, batch_size):
    train_dl = torch.utils.data.DataLoader(
        tokenized_datasets["train"],
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,
    )

    validation_dl = torch.utils.data.DataLoader(
        tokenized_datasets["validation"],
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
    )

    test_dl = torch.utils.data.DataLoader(
        tokenized_datasets["test"],
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
    )
    return train_dl, validation_dl, test_dl

In [ ]:
batch_size = 1

train_dl = torch.utils.data.DataLoader(
    ds_train_tokenized,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
)
validation_dl = torch.utils.data.DataLoader(
    ds_val_tokenized,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
)

## Training Custom loop

In [ ]:
N_EPOCHS = 3

In [ ]:
history = train_eval(
    train_dl,
    validation_dl,
    model=model,
    optimizer=optimizer,
    n_epochs=N_EPOCHS,
    save_name=f"test_model",
)
# history_list_c_lr3.append(history)

tensor [[utt + pad], [utt + pad], [utt + pad]]
tensor [CLS + utt + SEP + utt + SEP + utt + SEP + pad]
[ tensor[cls utt padd] tensor [cls utt padd]]


## Training con Classe Trainer

In [ ]:
class BertTrainer(Trainer):
    def __init__(self, model, training_args, train_ds, eval_ds, metrics):
        super().__init__(model, training_args, train_ds, eval_ds, metrics)

    def compute_loss(self, model, inputs, return_outputs=False):

        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        emotions_labels = inputs["emotions"]
        triggers_labels = inputs["triggers"]
        output_emotions, output_triggers = model(ids, mask)

        custom_loss = self.loss_fn(
            output_emotions, output_triggers, emotions_labels, triggers_labels
        )

        return (
            (custom_loss, output_emotions, output_triggers)
            if return_outputs
            else custom_loss
        )

    def loss_fn(outputs_emotions, outputs_triggers, emotions_labels, triggers_labels):
        return torch.nn.CrossEntropyLoss(
            outputs_emotions, emotions_labels
        ) + torch.nn.BCELoss(outputs_triggers, triggers_labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="\\test",
    do_train=True,
    do_eval=True,
    # evaluate_during_training=True,
    learning_rate=5e-5,
    num_train_epochs=8,
    seed=42,
)

In [ ]:
trainer = Trainer(
    model=model_full,
    args=training_args,
    # data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    train_dataset=ds_train,
    eval_dataset=ds_val,
    compute_metrics=sequence_f1,
    # tokenizer=tokenizer,
)

In [ ]:
trainer.train()